In [132]:
import re
import time


import nltk

nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/fagh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [133]:
raw = [
    'Run a marathon in under two hours. Impossible? Not for Nike (@Nike). Last May, the company brought three of the best runners on the planet together in Italy to set a new record in a closed-door marathon that was broadcast live on Twitter.'
    ,
    'For Pandora (@pandoramusic), there is bound to be a piece of music for each emoji. At least this is what the American music streaming giant wanted to prove with its campaign #SoundsLike.']
import pandas as pd

df = pd.DataFrame(raw)
df.head()

,0
0,Run a marathon in under two hours. Impossible?...
1,"For Pandora (@pandoramusic), there is bound to..."


In [146]:
tag_list = ['DeepstateVirus', 'DeepStateVaccine', 'DeepStateFauci', 'QAnon', 'Agenda21', 'CCPVirus',
            'ClimateChangeHoax', 'GlobalWarmingHoax', 'ChinaLiedPeopleDied', 'SorosVirus', '5GCoronavirus', 'MAGA',
            'WWG1WGA', 'Chemtrails', 'flatEarth', 'MoonLandingHoax', 'moonhoax', 'illuminati', 'pizzaGateIsReal',
            'PedoGateIsReal', '911truth', '911insidejob', 'reptilians']

tag_list_lower = list(map(lambda x: x.lower(), tag_list))



In [135]:
sid.polarity_scores(df[0][0])
# df['scores'] = df[0].apply(lambda review: sid.polarity_scores(review))

# df.head()

{'neg': 0.0, 'neu': 0.903, 'pos': 0.097, 'compound': 0.6369}

In [136]:
from psycopg2.extras import RealDictCursor
import psycopg2

""" Connect to the PostgreSQL database server """
conn = None


def exec_sql(sql: str, update_popici_hack_Data = None):
    try:
        conn = psycopg2.connect("dbname=data user=postgres password=root host=172.23.0.2")

        # create a cursor
        cur = conn.cursor(cursor_factory=RealDictCursor)
        # cur = conn.cursor()

        # execute a statement
        cur.execute(sql, update_popici_hack_Data)
        if update_popici_hack_Data is not None:
            conn.commit()
            result = cur.rowcount
        else:
            result = cur.fetchall()
        # close the communication with the PostgreSQL
        cur.close()
        return result
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            # print('Database connection closed.')


In [137]:
# for i in tag_list_lower:
#     sql = INSERT INTO public.hoaxs (id, name) VALUES (null, 'aaa')

In [147]:
tag_list_ids = []
for hoax in tag_list_lower:
    sql = f'select * from hashtags where LOWER(value) like \'%{hoax}%\''
    rows = exec_sql(sql)
    tag_list_ids.extend(map(lambda x: x['id'], rows))


In [148]:
sql = f'select DISTINCT tweet_hashtags.tweet_id from  tweet_hashtags inner join tweets t on tweet_hashtags.tweet_id = t.id where tweet_hashtags.hashtag_id in {str(tuple(tag_list_ids))} and t.neg is null'
rows = exec_sql(sql)

ids_to_process = list(map(lambda x: x['tweet_id'], rows))
print('ok')




ok


In [ ]:
def filter_content(content):
    content = re.sub(r'\s?[@#]\w*', '', content)
    content = re.sub(r'\s?https:\/\/t\.co\/\w*', '', content)
    emoj = re.compile("\s?["
                      u"\U0001F600-\U0001F64F"  # emoticons
                      u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                      u"\U0001F680-\U0001F6FF"  # transport & map symbols
                      u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                      u"\U00002500-\U00002BEF"  # chinese char
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"  # dingbats
                      u"\u3030"
                      "]+", re.UNICODE)
    content = emoj.sub(r'', content)
    return content

In [144]:
def init_worker(function):
    function.cursor = db.conn()

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


KeyboardInterrupt: 

In [ ]:
import time
from multiprocessing import Pool
start = time.time()

def update_vader(processing_id):
    sql = f'select content from tweets where id = \'{processing_id}\''
    rows = exec_sql(sql)



    # print(processing_id)
    # print('***************************')
    # print(rows[0]['content'])
    # print('----------------------------')
    x = filter_content(rows[0]['content'])
    vader = sid.polarity_scores(x)
    sql = 'UPDATE tweets SET neg = %s, pos = %s, neu = %s, compound = %s WHERE id = %s'
    data = [vader["neg"], vader["pos"], vader["neu"], vader["compound"], processing_id]

    exec_sql(sql,data)
    # return 'ok'

with Pool(processes=4) as pool:
    for i in pool.imap_unordered(update_vader, ids_to_process):
        pass
print(time.time()- start)
